In [ ]:
! pip install datasets langchain_community sentence_transformers chromadb langchain_openai ragas==0.0.19 --q
#! pip install transformers[torch] --q
#! pip install accelerate==0.27.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
import pickle
with open('data.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
#data

In [ ]:
from google.colab import userdata
import pandas as pd
from datasets import Dataset
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

In [ ]:
import pandas as pd
df = pd.read_csv('sampled_500_test_set.csv')

In [ ]:
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(df, page_content_column="context")

In [ ]:
docs = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=8000,
                                      chunk_overlap=0,
                                      length_function=len
                                      )

chunks = text_splitter.split_documents(documents=docs)

In [ ]:
len(chunks)

500

In [ ]:
def load_chroma(chunks, embedding_function):


    chroma_cliet = chromadb.Client()
    chroma_collection = chroma_cliet.create_collection(name='rag_ds', embedding_function=embedding_function)

    ids = [str(i) for i in range(len(chunks))]

    chroma_collection.add(ids=ids, documents=chunks)

    return chroma_collection

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings


model_name = 'aritrasen/bge-base-en-v1.5-finetuned_ragds'
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

from langchain_community.vectorstores import Chroma

### Creating Retriever using Vector DB

db = Chroma.from_documents(chunks, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 1})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/97.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini-2024-07-18')

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [ ]:
test_qa = df.loc[:,'question'].values

In [ ]:
ground_truths = []
for i in df.loc[:,'answer']:
    ground_truths.append([str(i)])

In [ ]:
len(test_qa) , len(ground_truths)

(500, 500)

In [ ]:
from tqdm import tqdm

In [ ]:
answers = []
contexts = []

for qns in tqdm(test_qa):
    answers.append(retrieval_chain.invoke(qns))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(qns)])

  0%|          | 0/500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
100%|██████████| 500/500 [09:07<00:00,  1.09s/it]


In [ ]:

# # To dict
# data = {
#     "question": test_qa,
#     "answer": answers,
#     "contexts": contexts,
#     "ground_truths": ground_truths
# }

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [ ]:
dataset.to_pandas().to_csv('finetuned_rag_eval_before_score.csv')

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    #context_utilization
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/57.4M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/517 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
metrices = [context_precision,context_recall,faithfulness,answer_relevancy]

In [ ]:
result_base = evaluate(
    dataset = dataset,
    metrics=metrices,
    #llm=llm,
    #embeddings=embeddings,
)

evaluating with [context_precision]


100%|██████████| 34/34 [04:36<00:00,  8.12s/it]


evaluating with [context_recall]


100%|██████████| 34/34 [14:28<00:00, 25.56s/it]


evaluating with [faithfulness]


100%|██████████| 34/34 [20:08<00:00, 35.54s/it]


evaluating with [answer_relevancy]


100%|██████████| 34/34 [15:47<00:00, 27.86s/it]


In [ ]:
result_base.to_pandas().to_csv('embedding_adp_rag_eval_score.csv')

In [ ]:
12

12